In [3]:
import joblib
import pandas as pd
import numpy as np
import re
from difflib import get_close_matches
from rapidfuzz import process
import datetime

# ✅ Load the trained model and encoders
import joblib
import requests
import io

# Raw GitHub URL (change "blob/main" to "raw/main")
url1 = "https://raw.githubusercontent.com/Deshan-Senanayake/Bird-Range-Prediction/main/Migration%20model/models/migration_prediction_model.pkl"

# Download the file
response1 = requests.get(url1)
response1.raise_for_status()  # Ensure we notice bad responses

# Load the model from memory
model_data1 = joblib.load(io.BytesIO(response1.content))

# Extract trained model and features
rf_model = model_data1['rf_final']
label_encoders1 = model_data1['label_encoders']
selected_features1 = model_data1['selected_features']


# ✅ Locality and Bird Name Handling
valid_localities = [
    "Buckingham Place Hotel Tangalle", "Bundala NP General", "Bundala National Park", 
    "Kalametiya", "Tissa Lake", "Yala National Park General", "Debarawewa Lake"
]

valid_bird_names = ["Blue-tailed Bee-eater", "Red-vented Bulbul", "White-throated Kingfisher"]

bird_aliases = {
    "blue tailed bird": "Blue-tailed Bee-eater",
    "blue bird": "Blue-tailed Bee-eater",
    "bee eater": "Blue-tailed Bee-eater",
    "red bird": "Red-vented Bulbul",
    "bulbul": "Red-vented Bulbul",
    "white bird": "White-throated Kingfisher",
    "kingfisher": "White-throated Kingfisher"
}

# ✅ Function: Handle Bird Name Variations
def correct_bird_name(name):
    name = name.lower()
    if name in bird_aliases:
        return bird_aliases[name]
    matches = get_close_matches(name, [b.lower() for b in valid_bird_names], n=1, cutoff=0.3)
    if matches:
        return next(b for b in valid_bird_names if b.lower() == matches[0])
    return "Unknown Bird"

# ✅ Function: Handle Locality Variations
def correct_locality(user_input):
    """
    Uses keyword-based mapping to handle user input more flexibly.
    """
    user_input = user_input.lower()

    # ✅ 1. Check for exact match first
    for loc in valid_localities:
        if user_input == loc.lower():
            return loc

    # ✅ 2. Check if user input is a **partial match** (keyword-based)
    for loc in valid_localities:
        if user_input in loc.lower():
            return loc  # Return the first match

    # ✅ 3. Handle some **manual keyword mappings** for better accuracy
    manual_mappings = {
        "bundala": "Bundala NP General",
        "yala": "Yala National Park General",
        "tissa": "Tissa Lake",
        "debara": "Debarawewa Lake",
        "kalametiya": "Kalametiya Bird Sanctuary"
    }
    
    for keyword, mapped_location in manual_mappings.items():
        if keyword in user_input:
            return mapped_location

    return "Unknown Location"  # No match found


# ✅ Function: Convert Day Name to Integer
def day_name_to_int(day_name):
    days_map = {
        "monday": 0, "tuesday": 1, "wednesday": 2, "thursday": 3, 
        "friday": 4, "saturday": 5, "sunday": 6
    }
    return days_map.get(day_name.lower(), None)

# ✅ Function: Convert Time of Day to Hour Range
def time_of_day_to_hour(time_str):
    time_ranges = {
        "morning": (6, 10), "afternoon": (11, 15),
        "evening": (16, 19), "night": (20, 23)
    }
    return time_ranges.get(time_str.lower(), None)

# ✅ Function: Extract Features from Query
import datetime

def extract_query_features_bird_presence(query1):
    query1 = query1.lower()
    
    # Extract Year
    year_match = re.search(r'\b(20[0-9]{2})\b', query1)
    year = int(year_match.group()) if year_match else None

    # Extract Month
    months_map = {
        "january": 1, "february": 2, "march": 3, "april": 4, "may": 5, "june": 6,
        "july": 7, "august": 8, "september": 9, "october": 10, "november": 11, "december": 12
    }
    month_match = re.search(r'\b(january|february|march|april|may|june|july|august|september|october|november|december)\b', query1)
    month = months_map.get(month_match.group()) if month_match else None

    # Extract Day of Week
    day_name_match = re.search(r'\b(monday|tuesday|wednesday|thursday|friday|saturday|sunday)\b', query1)
    day_of_week = day_name_to_int(day_name_match.group()) if day_name_match else None

    # Extract Time (Hour)
    time_match = re.search(r'\b([0-9]{1,2}):?([0-9]{2})?\s?(am|pm)?\b', query1)
    if time_match:
        hour = int(time_match.group(1))
        if time_match.group(3) == "pm" and hour < 12:
            hour += 12
    else:
        time_match = re.search(r'\b(morning|afternoon|evening|night)\b', query1)
        hour_range = time_of_day_to_hour(time_match.group()) if time_match else None
        hour = hour_range[0] if hour_range else None

    # Extract Locality
    locality_match = re.search(r'\b[a-zA-Z\s]+\b', query1)
    locality = correct_locality(locality_match.group()) if locality_match else None

    # Extract Bird Name
    bird_name_match = re.search(r'\b(?:' + '|'.join([b.lower().replace("-", ".*") for b in valid_bird_names]) + r')\b', query1)
    bird_name = correct_bird_name(bird_name_match.group()) if bird_name_match else None


    for alias, correct_name in bird_aliases.items():
        if alias in query1:
            bird_name = correct_name  # Use mapped alias name
            break  # Stop after the first match

    missing_inputs = []

    if year is None:
        year = 2025
        print("⚠️ Year is necessary to run the model. Since you didn't input it, year is 2025 by default. If you want another year, please enter it:")
        new_year = input().strip()
        if new_year.isdigit():
            year = int(new_year)

    if month is None:
        month = datetime.datetime.now().month  # Get current month
        print(f"⚠️ Month is necessary to run. Since you didn't input it, month is {month} by default. If you want another month, please enter it:")
        new_month = input().strip().lower()
        if new_month in months_map:
            month = months_map[new_month]

    if day_of_week is None:
        day_of_week = datetime.datetime.today().weekday()  # Get current day of the week
        print(f"⚠️ A day in the week is necessary to run. Since you didn't input it, day of week is {list(day_name_to_int.keys())[day_of_week].capitalize()} by default. If you want another day, please enter it (Monday-Sunday):")
        new_day = input().strip().lower()
        if new_day in day_name_to_int.keys():
            day_of_week = day_name_to_int(new_day)

    if hour is None:
        hour = datetime.datetime.now().hour  # Get current hour
        print(f"⚠️ Hour or a time period is necessary to run. Since you didn't input it, hour is {hour} by default. If you want another hour or a time period (morning, day, afternoon, evening), please enter it:")
        new_hour = input().strip().lower()
        if new_hour.isdigit():
            hour = int(new_hour)
        elif new_hour in time_of_day_to_hour.keys():
            hour = time_of_day_to_hour[new_hour][0]

    if locality is None or locality == "Unknown Location":
        print("⚠️ A location should be entered to run the models. Please select a location from the list below:")
        print("\n".join(valid_localities))
        locality = input("Enter the correct locality: ").strip()
        locality = correct_locality(locality)

    if bird_name is None or bird_name == "Unknown Bird":
        print("⚠️ A bird species should be entered to run the models. Please select a bird species from the list below:")
        print("\n".join(valid_bird_names))
        bird_name = input("Enter the correct bird species: ").strip()
        bird_name = correct_bird_name(bird_name)

    return {
        "year": year,
        "month": month,
        "day_of_week": day_of_week,
        "hour": hour,
        "locality": locality,
        "bird_name": bird_name
    }


def generate_meaningful_query(result1, original_query):
    """
    Converts the structured prediction result into a meaningful sentence.
    Ensures the time of day matches the original query.
    """
    if "error" in result1:
        return result1["error"]

    presence_text = "is likely to be present" if result1["predicted_presence"] == 1 else "is unlikely to be present"
    
    # Convert day index back to string
    days_map = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    day_name = days_map[result1["features_used"]["day_of_week"]]

    # Extract original time phrase from query (morning, afternoon, evening, night)
    time_of_day_match = re.search(r'\b(morning|afternoon|evening|night)\b', original_query.lower())
    if time_of_day_match:
        time_of_day = time_of_day_match.group()  # Get the exact phrase
    else:
        # Convert hour into a more natural phrase if no explicit time was mentioned
        hour = result1["features_used"]["hour"]
        if 6 <= hour <= 10:
            time_of_day = "in the morning"
        elif 11 <= hour <= 15:
            time_of_day = "in the afternoon"
        elif 16 <= hour <= 19:
            time_of_day = "in the evening"
        elif 20 <= hour <= 23:
            time_of_day = "at night"
        else:
            time_of_day = f"at {hour}:00"  # Show exact time if out of range

    probability = result1["probability"] * 100  # Convert to percentage

    # Construct a meaningful sentence
    query_sentence = (
        f"The {result1['features_used']['bird_name']} {presence_text} "
        f"at {result1['features_used']['locality']} on {day_name}, {result1['features_used']['month']}/"
        f"{result1['features_used']['year']} {time_of_day}. (Confidence: {probability:.1f}%)"
    )

    return query_sentence

# ✅ Function: Predict Bird Presence
def predict_bird_presence(query1):
    features1 = extract_query_features_bird_presence(query1)
    
    # ✅ Ensure `features` is a dictionary
    if not isinstance(features1, dict):
        return {"error": "Failed to extract features. Please check your query format."}

    # ✅ If the extracted features contain errors, return immediately
    if "error" in features1:
        return features1

    # ✅ Handle Unknown Locality
    if features1["locality"] == "Unknown Location":
        return {"error": "The locality entered is not recognized. Please provide a more specific location."}
    
    # ✅ Ensure bird name is correctly formatted (fixes case issue)
    features1["bird_name"] = next((b for b in valid_bird_names if b.lower() == features1["bird_name"].lower()), features1["bird_name"])

    # ✅ Get all known bird names from the encoder
    known_bird_names = set(label_encoders1['COMMON NAME'].classes_)

    # ✅ Check if the bird name exists in known labels
    if features1["bird_name"] not in known_bird_names:
        print(f"⚠️ Warning: '{features1['bird_name']}' is not in the trained bird name labels.")
        # ✅ Find the closest valid bird name (fuzzy matching)
        closest_match = get_close_matches(features1["bird_name"], known_bird_names, n=1, cutoff=0.3)
        if closest_match:
            print(f"✅ Using closest match: {closest_match[0]}")
            features1["bird_name"] = closest_match[0]
        else:
            return {"error": f"'{features1['bird_name']}' is not recognized. Please check the bird name."}

    try:
        # Encode locality and bird name
        locality_encoded = label_encoders1['LOCALITY'].transform([features1["locality"]])[0]
        bird_name_encoded = label_encoders1['COMMON NAME'].transform([features1["bird_name"]])[0]

        # Prepare input data
        input_data = pd.DataFrame([[features1["year"], features1["month"], features1["day_of_week"],
                                     features1["hour"], locality_encoded, bird_name_encoded]],
                                  columns=selected_features1)

        # Make prediction
        probability = rf_model.predict_proba(input_data)[:, 1][0]
        prediction = int(probability >= 0.5)

        # Structure the result
        result1 = {
            "query1": query1,
            "predicted_presence": prediction,
            "probability": round(probability, 3),
            "features_used": features1
        }

        return generate_meaningful_query(result1, query1)  # Pass original query

    except Exception as e:
        return {"error": f"Prediction error: {str(e)}"}


import joblib
import pandas as pd
import numpy as np
import re
import datetime
from difflib import get_close_matches

url2 = "https://raw.githubusercontent.com/Deshan-Senanayake/Bird-Range-Prediction/main/Migration%20model/models/location_prediction_model.pkl"


# Download the file
response2 = requests.get(url2)
response2.raise_for_status()  # Ensure we notice bad responses

# Load the model from memory
model_data2 = joblib.load(io.BytesIO(response2.content))

location_model = model_data2['location_model']
selected_features2 = model_data2['selected_features']
label_encoders2 = model_data2['label_encoders']

# ✅ Predefined Latitude & Longitude Values (Expand as Needed)
predefined_locations = [
    {"LATITUDE": 6.0463438, "LONGITUDE": 80.8541554},
    {"LATITUDE": 6.188598, "LONGITUDE": 81.2200356},
    {"LATITUDE": 6.1963995, "LONGITUDE": 81.2109113},
    {"LATITUDE": 6.1930548, "LONGITUDE": 81.2218203},
    {"LATITUDE": 6.0906125, "LONGITUDE": 80.9354124},
    {"LATITUDE": 6.188598, "LONGITUDE": 81.2200356},
    {"LATITUDE": 6.1930548, "LONGITUDE": 81.2218203}
]

# ✅ Bird Name Corrections (Handles NLP Variations)
bird_aliases = {
    "blue tailed bird": "Blue-tailed Bee-eater",
    "blue bird": "Blue-tailed Bee-eater",
    "bee eater": "Blue-tailed Bee-eater",
    "red bird": "Red-vented Bulbul",
    "bulbul": "Red-vented Bulbul",
    "white bird": "White-throated Kingfisher",
    "kingfisher": "White-throated Kingfisher"
}

valid_bird_names = ["Blue-tailed Bee-eater", "Red-vented Bulbul", "White-throated Kingfisher"]

# ✅ Function: Correct Bird Name with NLP
def correct_bird_name(name):
    name = name.lower()
    if name in bird_aliases:
        return bird_aliases[name]
    matches = get_close_matches(name, [b.lower() for b in valid_bird_names], n=1, cutoff=0.3)
    if matches:
        return next(b for b in valid_bird_names if b.lower() == matches[0])
    return "Unknown Bird"

# ✅ Function: Convert Day Name to Integer
def day_name_to_int(day_name):
    days_map = {
        "monday": 0, "tuesday": 1, "wednesday": 2, "thursday": 3, 
        "friday": 4, "saturday": 5, "sunday": 6
    }
    return days_map.get(day_name.lower(), None)

# ✅ Function: Convert Time of Day to Hour Range
def time_of_day_to_hour(time_str):
    time_ranges = {
        "morning": (6, 10), "afternoon": (11, 15),
        "evening": (16, 19), "night": (20, 23)
    }
    return time_ranges.get(time_str.lower(), None)


# ✅ Bird Name Aliases
bird_aliases = {
    "blue tailed bird": "Blue-tailed Bee-eater",
    "blue bird": "Blue-tailed Bee-eater",
    "bee eater": "Blue-tailed Bee-eater",
    "red bird": "Red-vented Bulbul",
    "bulbul": "Red-vented Bulbul",
    "white bird": "White-throated Kingfisher",
    "kingfisher": "White-throated Kingfisher"
}

def extract_query_features_location(query):
    query = query.lower()
    
    # Extract Year
    year_match = re.search(r'\b(20[0-9]{2})\b', query)
    year = int(year_match.group()) if year_match else None

    # Extract Month
    months_map = {
        "january": 1, "february": 2, "march": 3, "april": 4, "may": 5, "june": 6,
        "july": 7, "august": 8, "september": 9, "october": 10, "november": 11, "december": 12
    }
    month_match = re.search(r'\b(january|february|march|april|may|june|july|august|september|october|november|december)\b', query)
    month = months_map.get(month_match.group()) if month_match else None

    # Extract Day of Week
    day_name_match = re.search(r'\b(monday|tuesday|wednesday|thursday|friday|saturday|sunday)\b', query)
    day_of_week = day_name_to_int(day_name_match.group()) if day_name_match else None

    # Extract Time (Hour)
    time_match = re.search(r'\b([0-9]{1,2}):?([0-9]{2})?\s?(am|pm)?\b', query)
    if time_match:
        hour = int(time_match.group(1))
        if time_match.group(3) == "pm" and hour < 12:
            hour += 12
    else:
        time_match = re.search(r'\b(morning|afternoon|evening|night)\b', query)
        hour_range = time_of_day_to_hour(time_match.group()) if time_match else None
        hour = hour_range[0] if hour_range else None

    # Extract Bird Name (Allowing Both Alias & Actual Name)
    bird_name_match = re.search(r'\b(?:' + '|'.join([b.lower().replace("-", ".*") for b in valid_bird_names]) + r')\b', query)
    bird_name = bird_name_match.group() if bird_name_match else None

    # ✅ First, Check for Bird Alias Matches
    for alias, correct_name in bird_aliases.items():
        if alias in query:
            bird_name = correct_name  # Use mapped alias name
            break  # Stop after the first match

    # ✅ If No Alias Match, Ensure Actual Bird Name is in Correct Format
    if bird_name in valid_bird_names:
        bird_name = next(b for b in valid_bird_names if b.lower() == bird_name.lower())

    # ✅ Handle Missing Values With Default Assignments
    missing_inputs = []

    if year is None:
        year = 2025
        print("⚠️ Year is necessary to run the model. Since you didn't input it, year is 2025 by default. If you want another year, please enter it:")
        new_year = input().strip()
        if new_year.isdigit():
            year = int(new_year)

    if month is None:
        month = datetime.datetime.now().month  # Get current month
        print(f"⚠️ Month is necessary to run. Since you didn't input it, month is {month} by default. If you want another month, please enter it:")
        new_month = input().strip().lower()
        if new_month in months_map:
            month = months_map[new_month]

    if day_of_week is None:
        day_of_week = datetime.datetime.today().weekday()  # Get current day of the week
        print(f"⚠️ A day in the week is necessary to run. Since you didn't input it, day of week is {list(day_name_to_int.keys())[day_of_week].capitalize()} by default. If you want another day, please enter it (Monday-Sunday):")
        new_day = input().strip().lower()
        if new_day in day_name_to_int.keys():
            day_of_week = day_name_to_int(new_day)

    if hour is None:
        hour = datetime.datetime.now().hour  # Get current hour
        print(f"⚠️ Hour or a time period is necessary to run. Since you didn't input it, hour is {hour} by default. If you want another hour or a time period (morning, day, afternoon, evening), please enter it:")
        new_hour = input().strip().lower()
        if new_hour.isdigit():
            hour = int(new_hour)
        elif new_hour in time_of_day_to_hour.keys():
            hour = time_of_day_to_hour[new_hour][0]

    if bird_name is None or bird_name == "Unknown Bird":
        print("⚠️ A bird species should be entered to run the models. Please select a bird species from the list below:")
        print("\n".join(valid_bird_names))
        bird_name = input("Enter the correct bird species: ").strip()
        bird_name = correct_bird_name(bird_name)

    return {
        "year": year,
        "month": month,
        "day_of_week": day_of_week,
        "hour": hour,
        "bird_name": bird_name
    }


# ✅ Function: Predict Best Locations for Birdwatching
def predict_best_locations(query):
    features = extract_query_features_location(query)

    if "error" in features:
        return features

    # ✅ Ensure bird name is correctly formatted (fixes case issue)
    features["bird_name"] = next((b for b in valid_bird_names if b.lower() == features["bird_name"].lower()), features["bird_name"])

    # ✅ Get all known bird names from the encoder
    known_bird_names = set(label_encoders2['COMMON NAME'].classes_)

    # ✅ Check if the bird name exists in known labels
    if features["bird_name"] not in known_bird_names:
        print(f"⚠️ Warning: '{features['bird_name']}' is not in the trained bird name labels.")
        # ✅ Find the closest valid bird name (fuzzy matching)
        closest_match = get_close_matches(features["bird_name"], known_bird_names, n=1, cutoff=0.3)
        if closest_match:
            print(f"✅ Using closest match: {closest_match[0]}")
            features["bird_name"] = closest_match[0]
        else:
            return {"error": f"'{features['bird_name']}' is not recognized. Please check the bird name."}

    results = []
    for location in predefined_locations:
        input_data = pd.DataFrame([[features["year"], features["month"], features["day_of_week"],
                                     features["hour"], location["LATITUDE"], 1, location["LONGITUDE"],
                                     label_encoders2['COMMON NAME'].transform([features["bird_name"]])[0]]],
                                  columns=selected_features2)

        predicted_location_encoded = location_model.predict(input_data)[0]
        predicted_location = label_encoders2['LOCALITY'].inverse_transform([predicted_location_encoded])[0]

        results.append(predicted_location)

    # ✅ Remove duplicate locations and print results
    unique_locations = list(set(results))



    return unique_locations


import joblib
import pandas as pd
import numpy as np
import re
import datetime
from difflib import get_close_matches
from rapidfuzz import process

url3 = "https://raw.githubusercontent.com/Deshan-Senanayake/Bird-Range-Prediction/main/Migration%20model/models/time_prediction_model.pkl"


# Download the file
response3 = requests.get(url3)
response3.raise_for_status()  # Ensure we notice bad responses

# Load the model from memory
model_data3 = joblib.load(io.BytesIO(response3.content))

month_model = model_data3['month_model']
hour_model = model_data3['hour_model']
selected_features = model_data3['selected_features']
label_encoders = model_data3['label_encoders']

valid_localities = [
    "Buckingham Place Hotel Tangalle", "Bundala NP General", "Bundala National Park", 
    "Kalametiya", "Tissa Lake", "Yala National Park General", "Debarawewa Lake"
]

valid_bird_names = ["Blue-tailed Bee-eater", "Red-vented Bulbul", "White-throated Kingfisher"]

bird_aliases = {
    "blue tailed bird": "Blue-tailed Bee-eater",
    "bee eater": "Blue-tailed Bee-eater",
    "blue bird": "Blue-tailed Bee-eater",
    "red bird": "Red-vented Bulbul",
    "bulbul": "Red-vented Bulbul",
    "white bird": "White-throated Kingfisher",
    "kingfisher": "White-throated Kingfisher"
}

season_aliases = {
    "summer": "Is_Summer",
    "winter": "Is_Winter",
    "spring": "Is_Spring",
    "autumn": "Is_Autumn"
}

time_period_aliases = {
    "morning": "Is_Morning",
    "afternoon": "Is_Afternoon",
    "evening": "Is_Evening",
    "night": "Is_Night"
}

# ✅ Function: Handle Bird Name Variations
def correct_bird_name(name):
    name = name.lower().strip()
    
    # ✅ Check in aliases first
    if name in bird_aliases:
        return bird_aliases[name]

    # ✅ Check for closest match in valid bird names
    matches = get_close_matches(name, [b.lower() for b in valid_bird_names], n=1, cutoff=0.3)
    if matches:
        return next(b for b in valid_bird_names if b.lower() == matches[0])

    return "Unknown Bird"

# ✅ Function: Convert Day Name to Integer
def day_name_to_int(day_name):
    days_map = {
        "monday": 0, "tuesday": 1, "wednesday": 2, "thursday": 3, 
        "friday": 4, "saturday": 5, "sunday": 6
    }
    return days_map.get(day_name.lower(), None)

# ✅ Function: Handle Locality Variations
def correct_locality(user_input):
    user_input = user_input.lower().strip()

    # ✅ Exact match
    for loc in valid_localities:
        if user_input == loc.lower():
            return loc

    # ✅ Partial match
    for loc in valid_localities:
        if user_input in loc.lower():
            return loc

    # ✅ Manual mappings
    manual_mappings = {
        "bundala": "Bundala NP General",
        "yala": "Yala National Park General",
        "tissa": "Tissa Lake",
        "debara": "Debarawewa Lake",
        "kalametiya": "Kalametiya Bird Sanctuary"
    }

    for keyword, mapped_location in manual_mappings.items():
        if keyword in user_input:
            return mapped_location

    return "Unknown Location"

# ✅ Function: Extract Features from Query
def extract_query_features_time(query):
    query = query.lower().strip()
    
    # Extract Year
    year_match = re.search(r'\b(20[0-9]{2})\b', query)
    year = int(year_match.group()) if year_match else 2025  # ✅ Default to 2025
    if not year_match:
        print("⚠️ Year is necessary to run the model. Since you didn't input it, year is 2025 by default. If you want another year, please enter it:")
        new_year = input().strip()
        if new_year.isdigit():
            year = int(new_year)

    # Extract Day of Week
    day_name_match = re.search(r'\b(monday|tuesday|wednesday|thursday|friday|saturday|sunday)\b', query)
    day_of_week = day_name_to_int(day_name_match.group()) if day_name_match else datetime.datetime.today().weekday()
    if not day_name_match:
        print(f"⚠️ A day in the week is necessary to run. Since you didn't input it, day of week is {list(day_name_to_int.keys())[day_of_week].capitalize()} by default. If you want another day, please enter it (Monday-Sunday):")
        new_day = input().strip().lower()
        if new_day in day_name_to_int.keys():
            day_of_week = day_name_to_int(new_day)

    # Extract Bird Name
    bird_name = None
    for alias, correct_name in bird_aliases.items():
        if alias in query:
            bird_name = correct_name
            break
    if not bird_name:
        bird_name_match = re.search(r'\b(?:' + '|'.join([b.lower().replace("-", ".*") for b in valid_bird_names]) + r')\b', query)
        bird_name = bird_name_match.group() if bird_name_match else "Unknown Bird"

    if bird_name == "Unknown Bird":
        print("⚠️ A bird species should be entered to run the models. Please select a bird species:")
        print("\n".join(valid_bird_names))
        bird_name = input("Enter the correct bird species: ").strip()
        bird_name = correct_bird_name(bird_name)

    # Extract Locality
    locality_match = re.search(r'\b[a-zA-Z\s]+\b', query)
    locality = correct_locality(locality_match.group()) if locality_match else "Unknown Location"
    if locality == "Unknown Location":
        print("⚠️ A location should be entered to run the models. Please select a location:")
        print("\n".join(valid_localities))
        locality = input("Enter the correct locality: ").strip()
        locality = correct_locality(locality)

    # Extract Season & Time Period
    season_flags = {season: 0 for season in season_aliases.values()}
    time_period_flags = {time: 0 for time in time_period_aliases.values()}

    for season, flag in season_aliases.items():
        if season in query:
            season_flags[flag] = 1

    for time, flag in time_period_aliases.items():
        if time in query:
            time_period_flags[flag] = 1

    if sum(season_flags.values()) == 0:
        print("⚠️ The season is necessary. Please select one:\nSummer\nWinter\nSpring\nAutumn")
        user_season = input("Enter the season: ").strip().lower()
        if user_season in season_aliases:
            season_flags[season_aliases[user_season]] = 1

    if sum(time_period_flags.values()) == 0:
        print("⚠️ The time period is necessary. Please select one:\nMorning\nAfternoon\nEvening\nNight")
        user_time_period = input("Enter the time period: ").strip().lower()
        if user_time_period in time_period_aliases:
            time_period_flags[time_period_aliases[user_time_period]] = 1

    return {
        "year": year,
        "day_of_week": day_of_week,
        "locality": locality,
        "bird_name": bird_name,
        **season_flags,
        **time_period_flags
    }

# ✅ Function: Predict Best Time for Birdwatching (Same as before, no changes needed)


# ✅ Function: Predict Best Time for Birdwatching
# ✅ Function: Predict Best Time for Birdwatching
def predict_best_time(query):
    features = extract_query_features_time(query)

    if "error" in features:
        return features

    # ✅ Ensure bird name is correctly formatted (fixes case issue)
    features["bird_name"] = next((b for b in valid_bird_names if b.lower() == features["bird_name"].lower()), features["bird_name"])

    # ✅ Get all known bird names from the encoder
    known_bird_names = set(label_encoders['COMMON NAME'].classes_)

    # ✅ Check if the bird name exists in known labels
    if features["bird_name"] not in known_bird_names:
        print(f"⚠️ Warning: '{features['bird_name']}' is not in the trained bird name labels.")
        # ✅ Find the closest valid bird name (fuzzy matching)
        closest_match = get_close_matches(features["bird_name"], known_bird_names, n=1, cutoff=0.3)
        if closest_match:
            print(f"✅ Using closest match: {closest_match[0]}")
            features["bird_name"] = closest_match[0]
        else:
            return {"error": f"'{features['bird_name']}' is not recognized. Please check the bird name."}

    try:
        # ✅ Encode locality and bird name
        locality_encoded = label_encoders['LOCALITY'].transform([features["locality"]])[0]
        bird_name_encoded = label_encoders['COMMON NAME'].transform([features["bird_name"]])[0]

        # ✅ Prepare Input Data
        input_data = pd.DataFrame([[1, features["year"], features["day_of_week"],
                                    locality_encoded, bird_name_encoded, 
                                    features["Is_Summer"], features["Is_Winter"], features["Is_Spring"], features["Is_Autumn"],
                                    features["Is_Morning"], features["Is_Afternoon"], features["Is_Evening"], features["Is_Night"]]],
                                columns=selected_features)

        # ✅ Predict Best Month & Hour
        predicted_month = int(round(month_model.predict(input_data)[0]))
        predicted_hour = int(round(hour_model.predict(input_data)[0]))

        # ✅ Convert Month Number to Month Name
        months_map = {
            1: "January", 2: "February", 3: "March", 4: "April", 5: "May", 6: "June",
            7: "July", 8: "August", 9: "September", 10: "October", 11: "November", 12: "December"
        }
        month_name = months_map.get(predicted_month, f"Unknown ({predicted_month})")

        # ✅ Convert Hour to AM/PM Format
        am_pm = "a.m." if predicted_hour < 12 else "p.m."
        formatted_hour = predicted_hour if predicted_hour <= 12 else predicted_hour - 12
        if formatted_hour == 0:
            formatted_hour = 12  # Convert 0-hour to 12 AM

        # ✅ Return Formatted Output
        return (
            f"✅ Best Time for Birdwatching:\n"
            f"📅 Month: {month_name} ({predicted_month})\n"
            f"⏰ Hour: {formatted_hour}:00 {am_pm}"
        )

    except Exception as e:
        return {"error": f"Prediction error: {str(e)}"}


import sys

if __name__ == "__main__":
    print("\n📢 Welcome to the Birdwatching Assistant!")
    print("Choose a query type:")
    print("1️⃣ Bird Presence Prediction")
    print("2️⃣ Best Locations for Birdwatching")
    print("3️⃣ Best Time for Birdwatching")
    print("Type 'exit' or 'quit' to stop.\n")

    while True:
        sys.stdin.flush()  # ✅ Clear input buffer to fix delayed inputs
        option = input("🔹 Enter your choice (1/2/3): ").strip()

        if option.lower() in ["exit", "quit"]:
            print("🚪 Exiting the system. Happy birdwatching!")
            break

        if option not in ["1", "2", "3"]:
            print("❌ Invalid choice. Please enter 1, 2, or 3.")
            continue

        sys.stdin.flush()  # ✅ Clear input buffer before taking query input
        query = input("🔍 Enter your query: ").strip()

        if query.lower() in ["exit", "quit"]:
            print("🚪 Exiting the system. Happy birdwatching!")
            break

        if option == "1":
            response = predict_bird_presence(query)
        elif option == "2":
            response = predict_best_locations(query)
        elif option == "3":
            response = predict_best_time(query)

        # ✅ Handle list output (for Option 2)
        if isinstance(response, list):
            print("✅ Recommended Locations for Birdwatching:\n")
            for idx, loc in enumerate(response, start=1):
                print(f"📍 Location {idx}: {loc}")
            print("\n")  # Extra line break for readability
        else:
            print(response + "\n")  # Normal string output




📢 Welcome to the Birdwatching Assistant!
Choose a query type:
1️⃣ Bird Presence Prediction
2️⃣ Best Locations for Birdwatching
3️⃣ Best Time for Birdwatching
Type 'exit' or 'quit' to stop.

The Blue-tailed Bee-eater is likely to be present at Bundala NP General on Friday, 4/2025 in the morning. (Confidence: 85.5%)

✅ Recommended Locations for Birdwatching:

📍 Location 1: Wewa  Bundala National ParkSouthern Province
📍 Location 2: Bundala Freshwater Marsh
📍 Location 3: Netolpitiya Southern Province
📍 Location 4: Kalametiya Bird Sanctuary
📍 Location 5: Bundala NP General


✅ Best Time for Birdwatching:
📅 Month: July (7)
⏰ Hour: 7:00 a.m.

🚪 Exiting the system. Happy birdwatching!


In [4]:
test_query = "Will I be able to see a blue tailed bird in Bundala on a Friday 10.00 a.m. in 2025 April?"
result = predict_bird_presence(test_query)
print(result)


The Blue-tailed Bee-eater is likely to be present at Bundala NP General on Friday, 4/2025 in the morning. (Confidence: 85.5%)


In [5]:
test_query2 = "Where can I find a Blue tailed bee eater on 2025 Thursday Morning in January?"
result2 = predict_best_locations(test_query2)


In [6]:
test_query = "When is the best time to see a Blue tailed bird in Bundala during a summer morning on sunday in 2025?"
result = predict_best_time(test_query)
print(result)

✅ Best Time for Birdwatching:
📅 Month: July (7)
⏰ Hour: 7:00 a.m.
